In [ ]:
from google.colab import files
import zipfile
import os

# Upload do ZIP
uploaded = files.upload()

# Extrair o ZIP
with zipfile.ZipFile("Projeto_HOG_SVM_ENTREGA.zip", "r") as zip_ref:
    zip_ref.extractall("Projeto_HOG_SVM_ENTREGA")

print("✅ ZIP extraído com sucesso!")


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Caminho para dataset
dataset_path = '/content/Projeto_HOG_SVM_ENTREGA/coffee_dataset_corrigido/train'
classes = ['arabica', 'robusta']


In [ ]:
X_hog = []
y = []

hog = cv2.HOGDescriptor()

for label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_path, class_name)
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 128))
            features = hog.compute(img)
            X_hog.append(features.flatten())
            y.append(label)


In [ ]:
X = np.array(X_hog)
y = np.array(y)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

model = SVC(kernel="linear")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Acurácia: {:.2f}%".format(acc * 100))

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, cmap="Blues")
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()
